In [ ]:
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

import itertools
import multiprocessing
from multiprocessing.dummy import Pool as ThreadPool
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
# Apparently, SNS stands for "Samuel Norman Seaborn", a fictional
# character from The West Wing
import seaborn as sns
import sympy

sns.set()
sympy.init_printing()
# Make the figures directory if it doesn't exist.
Path('figures/').mkdir(exist_ok=True)

In [ ]:
def generate_cities(n, scale=100):
    """Generates an array of cities of the given size.

    Pick random coordinates in the `scale`x`scale` grid uniformly.

    :param n: The size of the individual to generate.
    :param scale: How much to scale the individual's coordinates by.
    """
    # Scale up the uniform values from [0, 1].
    return np.random.rand(n, 2) * scale

def generate_population(cities, size, scale=100):
    """Generate a population of individuals with the given size.

    Each individual is an array of indices into the given cities array.

    :param cities: An unsorted array of city locations.
    :param size: The number of individuals to generate.
    :param scale: How much to scale the individual's coordinates by.
    """
    n = len(cities)
    population = np.zeros((size, n), dtype=int)
    for i in range(size):
        individual = np.arange(n, dtype=int)
        np.random.shuffle(individual)
        population[i] = individual
    return population

In [ ]:
cities = generate_cities(50)
# Generate one random path
population = generate_population(cities, 1)
path = population[0]
plt.plot(cities[path][:, 0], cities[path][:, 1], 'r')
plt.plot(cities[:, 0], cities[:, 1], 'o')
plt.title('A random individual')
# plt.axis('equal')
plt.axis('scaled')
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.savefig('figures/prob2-random-individual.pdf')
plt.show()

In [ ]:
def pairwise(iterable):
    """Iterate over the given iterable in pairs.

    pairwise([1, 2, 3, 4]) -> (1, 2), (2, 3), (3, 4)
    """
    a, b = itertools.tee(iterable)
    # Advance b one step
    next(b, None)
    return zip(a, b)

In [ ]:
def fitness(cities, path):
    """Evaluate the fitness of the given path.

    Compute the Euclidean distance between every pair of cities in the path
    and add them together.

    :param cities: The array of cities through which to compute a path.
    :param path: The path through the given cities to compute the fitness for.
    :returns: The fitness of the individual.
    """
    individual = cities[path]
    return 1 / sum(np.linalg.norm(c1 - c2) for c1, c2 in pairwise(individual))

In [ ]:
def swap_mutation(path):
    """Swap two random cities in the path.

    Returns a new mutated copy of the given array.
    """
    # Arrays are (kind of) passed by reference in Python.
    x = np.copy(path)
    # Generate two valid indices to swap.
    i = np.random.randint(0, len(x) - 2)
    j = np.random.randint(i, len(x) - 1)
    # array indexing returns views, not copies
    temp = np.copy(x[i])
    x[i] = x[j]
    x[j] = temp

    return x

In [ ]:
a = np.arange(10, dtype=int)
swap_mutation(a)

In [ ]:
def insertion_mutation(path):
    """Insert a random value in the list somewhere else in the list.

    Returns a new mutated copy of the given array.
    """
    i = np.random.randint(0, len(path) - 2)
    j = np.random.randint(i, len(path) - 1)
    # Delete the element at index j and insert it before index i.
    temp = np.delete(path, j, axis=0)
    temp = np.insert(temp, i, path[j], axis=0)

    return temp

In [ ]:
# print(a)
insertion_mutation(a)

In [ ]:
def displacement_mutation(path):
    """Inserts a random subarray in the list somewhere else.

    Returns a new mutated copy of the given array.
    """
    # Pick a random subarray
    i = np.random.randint(0, len(path) - 2)
    j = np.random.randint(i, len(path) - 1)
    subarray = path[i:j]
    # Delete the given subarray
    tmp = np.delete(path, range(i, j), axis=0)
    k = np.random.randint(0, len(tmp) - 1)

    return np.insert(tmp, k, subarray, axis=0)

In [ ]:
displacement_mutation(a)

In [ ]:
def shuffle_mutation(path):
    """Shuffles a random subarray in the given path.

    Returns a new mutated copy of the given array.
    """
    # Pick a random subarray
    i = np.random.randint(0, len(path) - 2)
    j = np.random.randint(i, len(path) - 1)
    x = np.copy(path)
    np.random.shuffle(x[i:j])

    return x

In [ ]:
shuffle_mutation(a)

In [ ]:
def inversion_mutation(path):
    """Inverts a random subarray in the given path.

    Returns a new mutated copy of the given array.
    """
    i = np.random.randint(0, len(path) - 2)
    j = np.random.randint(i, len(path) - 1)
    x = np.copy(path)
    # Invert the subarray.
    x[i:j] = x[i:j][::-1]

    return x

In [ ]:
inversion_mutation(a)

In [ ]:
def mutate(path, method='inversion'):
    """Mutate the given individual via the given method.

    Returns a new mutated copy of the given array.

    :param method: One of 'swap', 'insertion', 'displacement',
    'shuffle', or 'inversion'. Defaults to 'inversion'.
    """
    methods = {
        'swap': swap_mutation,
        'insertion': insertion_mutation,
        'displacement': displacement_mutation,
        'shuffle': shuffle_mutation,
        'inversion': inversion_mutation,
    }
    return methods[method](path)

In [ ]:
mutate(a)

In [ ]:
def deterministic_selection(population, size, func, cities):
    """Deterministically select the most fit from the given population.

    Use the given fitness function to rank the population, then pick
    the next `size` of the population to move on. This assumes that,
    for a problem without recombination, the mutated individuals have
    been mixed in with the original population.

    :param population: The population to cull.
    :param size: The desired size of the population.
    :param func: The fitness function to rank the population by.
    :param cities: The array of city locations.
    :returns: The culled population, sorted upwards in increasing fitness.
    """
    fitnesses = np.array([func(cities, p) for p in population])
    indices = np.argsort(fitnesses, axis=0)
    euthanize = len(population) - size
    return population[indices][euthanize:]

In [ ]:
cities = generate_cities(4)
p = generate_population(cities, 10)
deterministic_selection(p, 5, fitness, cities)

In [ ]:
def stochastic_selection(population, size, func, cities):
    """Randomly select the most fit from the given population.
    
    Select without replacement an individual with probability
    proportional to its fitness.
    
    :param population: The population to cull.
    :param size: The desired size of the population.
    :param func: The fitness function to rank the population by.
    :param cities: The array of city locations.
    :returns: The culled population, unsorted.
    """
    fitnesses = np.array([func(cities, p) for p in population])
    probabilities = fitnesses / np.sum(fitnesses)
    survivors = np.random.choice(len(population), size, replace=False, p=probabilities)
    return population[survivors]

In [ ]:
p = generate_population(cities, 10)
stochastic_selection(p, 5, fitness, cities)

In [ ]:
def select(population, size, func, cities, method='deterministic'):
    """Select the `size` most fit from the given population.

    :param population: The population to cull.
    :param size: The desired size of the population.
    :param func: The fitness function to rank the population by.
    :param cities: The array of city locations.
    :param method: One of 'deterministic' or 'stochastic'.
    :returns: The culled population, in arbitrary order.
    """
    methods = {
        'stochastic': stochastic_selection,
        'deterministic': deterministic_selection,
    }
    return methods[method](population, size, func, cities)

In [ ]:
p = generate_population(cities, 10)
select(p, 5, fitness, cities)

In [ ]:
def simple_ea(cities, size, func, iters, mutation='inversion', selection='deterministic'):
    """Run the standard evolutionary algorithm to solve the TSP.

    This implementation does not use recombination.

    :param cities: The array of city locations.
    :param size: The population size to use.
    :param func: The fitness function to use.
    :param iters: The number of iterations (generations) to run.
    :param mutation: The type of mutation to use. One of 'swap', 'insertion',
    'displacement', 'shuffle', or 'inversion'.
    :param selection: The type of selection to use. One of 'deterministic',
    or 'stochastic'.
    """
    n = len(cities)
    population = generate_population(cities, size)
    best_fitnesses = np.zeros(iters)
    best_individuals = np.zeros((iters, n), dtype=int)
    for i in range(iters):
        # Do not recombine population.
        mutations = np.array([mutate(p, method=mutation) for p in population], dtype=int)
        combined = np.concatenate((population, mutations))
        population = select(combined, size, func, cities, method=selection)
        fitnesses = np.array([func(cities, p) for p in population])
        best = fitnesses.argmax()
        best_fitnesses[i] = fitnesses[best]
        best_individuals[i] = population[best]

    return best_fitnesses, best_individuals

In [ ]:
def plot_summary(fitnesses, cities, paths, description=''):
    """Plot a summary of a given run of the simple_ea algorithm."""
    plt.plot(range(len(fitnesses)), fitnesses)
    plt.title('Population fitness over time')
    plt.xlabel('generation')
    plt.ylabel('fitness')
    plt.savefig(f'figures/prob2-fitness-{description}.pdf')
    plt.show()
    
    best = fitnesses.argmax()
    solution = cities[paths[best]]
    plt.plot(solution[:, 0], solution[:, 1], 'r')
    plt.plot(solution[:, 0], solution[:, 1], 'o')
    plt.title(f'The best {description} individual $f={fitnesses[best]:.5f}$')
    plt.axis('scaled')
    plt.xlabel('$x$')
    plt.ylabel('$y$')
    plt.savefig(f'figures/prob2-best-{description}.pdf')
    plt.show()

In [ ]:
N = 40
pop_size = 50
generations = 800
cities = generate_cities(N)
fitnesses, paths = simple_ea(cities, pop_size, fitness, generations, mutation='inversion', selection='stochastic')

plot_summary(fitnesses, cities, paths, description='stochastic')

fitnesses, paths = simple_ea(cities, pop_size, fitness, generations, mutation='inversion', selection='deterministic')

plot_summary(fitnesses, cities, paths, description='deterministic')

In [ ]:
def evaluate(tour: np.ndarray) -> float:
    """Evaluate the length of the given tour.

    Compute the Euclidean distance between every pair of cities in the tour
    and add them together.

    :param tour: An array of cities, where each city is n (x, y) pair.
    :type tour: np.ndarray with shape (n, 2)
    :return: The length of the tour.
    :rtype: float
    """
    return sum(np.linalg.norm(c1 - c2) for c1, c2 in pairwise(tour))


def perturb(x: np.ndarray) -> np.ndarray:
    """Perturb the given array.

    Perform a sublist inversion in the interior of the given array.

    :param x: The array to perturb. Is not modified.
    :type x: np.ndarray
    :return: A perturbed copy of the given array.
    :rtype: np.ndarray
    """
    # Compute two random indices, avoiding the endpoints.
    i = np.random.randint(0, len(x) - 2)
    j = np.random.randint(i, len(x) - 1)

    # Produce a copy of the given array and invert a random sublist inside.
    y = np.copy(x)
    y[i:j] = y[i:j][::-1]

    return y


def accept_solution(energy1: float, energy2: float, temperature: float) -> bool:
    """Determine whether to accept a solution given current and previous energies.

    :param energy1: The energy of the current solution.
    :type energy1: float
    :param energy2: The energy of the previous solution.
    :type energy2: float
    :param temperature: The current temperature of the whole system.
    :type temperature: float
    :return: Whether or not to accept the current solution.
    :rtype: bool
    """
    return np.random.random() < np.exp((energy1 - energy2) / temperature)


def simulated_annealing(cities: np.ndarray, temperature=800, cooling_factor=0.001) -> np.ndarray:
    """Run the simulated annealing algorithm to solve the TSP.

    :param cities: An array of (x, y) city coordinates.
    :type cities: np.ndarray
    :param temperature: The starting temperature of the system, defaults to 800
    :param temperature: int, optional
    :param cooling_factor: How quickly to cool the system, defaults to 0.001
    :param cooling_factor: float, optional
    :return: A locally optimal tour through the given array of cities.
    :rtype: np.ndarray
    """
    current = evaluate(cities)
    energies = [current]
    while temperature > 0.001:
        new_solution = perturb(cities)
        energy = evaluate(new_solution)
        if accept_solution(current, energy, temperature):
            cities = new_solution
            current = energy
            energies.append(current)

        temperature *= 1 - cooling_factor
    return cities, np.array(energies)

In [ ]:
solution, fitnesses = simulated_annealing(cities, temperature=100)
best = 1 / np.min(fitnesses)
# Adjust the fitnesses to the same scale as the evolutionary algorithm
plt.plot(range(len(fitnesses)), 1 / fitnesses)
plt.title('Simulated annealing fitness over time')
plt.xlabel('iteration')
plt.ylabel('fitness')
plt.savefig('figures/prob2-simulated-annealing-fitness.pdf')
plt.show()
    
# Plot the path with a low zorder so that the cities are drawn on top.
plt.plot(solution[:, 0], solution[:, 1], 'r')
plt.plot(solution[:, 0], solution[:, 1], 'o')
plt.title(f'The simulated annealing solution $f={best:.5f}$')
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.axis("scaled")
plt.savefig('figures/prob2-simulated-annealing.pdf')
plt.show()